### https://colab.research.google.com/github/nestauk/im-tutorials/blob/3-ysi-tutorial/notebooks/Web-Scraping/Web%20Scraping%20Tutorial.ipynb used as model for web scraping technique

## Import necessary packages

In [2]:
from IPython.core.display import display, HTML

import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, Comment
import matplotlib.pyplot as plt
import sys
import os
import time
from unidecode import unidecode
import math

## Write Function to Scrape Player Data based on Name

#### Proof of concept script (Will be turned into a function and automated in later stages)

In [3]:


### Import Jalen Brunson's NBA data

time.sleep(3)

# Brunson NBA stats url + Walker Kessler (low and high height and weight values)

brunson_url = 'https://www.basketball-reference.com/players/b/brunsja01.html'   # Test for smaller physical values
#brunson_url = 'https://www.basketball-reference.com/players/k/kesslwa01.html'    # Test for large physical values

# Use requests to retrieve data from a given URL
brunson_response = requests.get(brunson_url)

# Parse the whole HTML page using BeautifulSoup
brunson_soup = BeautifulSoup(brunson_response.text, 'html.parser')

# Title of the parsed page
brunson_soup.title.string

# Find instance of paragraph markers for height and weight scraping
paraMarkersBrunson = brunson_soup.find_all('p')

# Create a dictionary to cache all of the player statistics
brunsonInfoDict = {}


# Traverse the list to look for height, weight, and shooting hand data
for sectionNum in range(len(paraMarkersBrunson)):

    # Find Height data in cm's (standard for all player pages)
    if paraMarkersBrunson[sectionNum].string != None and "tall" in paraMarkersBrunson[sectionNum].string:
        # All centimeter height values for NBA players are in triple digits so the format is constant
        print("Height: ", paraMarkersBrunson[sectionNum].string[-13:-9].strip())
        brunsonInfoDict['Height'] = int(paraMarkersBrunson[sectionNum].string[-13:-9].strip())
        
    
    # Find Weight data in lbs (standard for all player pages)
    if paraMarkersBrunson[sectionNum].string != None and "weigh" in paraMarkersBrunson[sectionNum].string:
        # All kg weight values for NBA players are in double or triple digits
        weight = paraMarkersBrunson[sectionNum].string[paraMarkersBrunson[sectionNum].string.index('(')+1:paraMarkersBrunson[sectionNum].string.index(')')-3].strip()
        if weight[0] == '(':
            weight = weight[1:]
        brunsonInfoDict['Weight'] = int(weight.strip())
        
        # Passes 3 digit and 2 digit weight case in kg (Walker Kessler and Jalen Brunson)
        print("Weight: ", weight)
        
print(brunsonInfoDict)   # Height and Weight Successfully Scraped


# Attempt to extract data from the tables (Per 100 possesions table first)
brunson_soup_2 = BeautifulSoup(requests.get(brunson_url).content, "lxml")
brunson_soup_3 = BeautifulSoup("\n".join(brunson_soup_2.find_all(text=Comment)), "lxml")
brunson_per_poss_df = pd.read_html(str(brunson_soup_3.select_one("table#per_poss")))[0]
brunson_per_poss_df = brunson_per_poss_df.loc[:list(brunson_per_poss_df['Season']).index('Career')-1].dropna(axis=1, how='all')

display(brunson_per_poss_df) # Prints the full dataframe correctly

brunsonInfoDict['per_poss_df'] = brunson_per_poss_df

def firstIndWith(string,lst):
    for i in range(len(lst)):
        for term in lst[i].columns:
            print(term)
            if string in str(term):
                return i
                for _ in range(1000):
                    print('---')
    return -1

# Attempt to extract data from the tables
brunson_advanced_stat_df_full_lst = pd.read_html(str(brunson_soup_3.select("table")))
print(firstIndWith("VORP",brunson_advanced_stat_df_full_lst))
brunson_advanced_stat_df = brunson_advanced_stat_df_full_lst[firstIndWith("VORP",brunson_advanced_stat_df_full_lst)]

# Remove the double header structure and replace with only a single header
brunson_advanced_stat_df.columns = [x[1] for x in brunson_advanced_stat_df.columns]

# Eliminate totals rows at the bottom
brunson_advanced_stat_df = brunson_advanced_stat_df.loc[:list(brunson_advanced_stat_df['Season']).index('Career')-1].dropna(axis=1, how='all')

display(brunson_advanced_stat_df.head()) # Prints the full dataframe correctly

brunsonInfoDict['advanced_stat_df'] = brunson_advanced_stat_df



        
    


Height:  188
Weight:  86
{'Height': 188, 'Weight': 86}


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,2018-19,22.0,DAL,NBA,PG,73.0,38.0,1591.0,8.0,17.2,...,4.4,5.2,7.0,1.1,0.1,2.7,3.9,20.7,110.0,114.0
1,2019-20,23.0,DAL,NBA,PG,57.0,16.0,1022.0,8.6,18.3,...,5.3,6.3,8.9,1.0,0.2,3.1,3.5,22.0,114.0,114.0
2,2020-21,24.0,DAL,NBA,PG,68.0,12.0,1697.0,9.5,18.2,...,5.9,6.7,6.9,1.0,0.0,2.3,3.2,24.9,123.0,116.0
3,2021-22,25.0,DAL,NBA,SG,79.0,61.0,2524.0,10.1,20.1,...,5.3,6.2,7.5,1.3,0.1,2.5,3.0,25.6,118.0,112.0
4,2022-23,26.0,NYK,NBA,PG,68.0,68.0,2379.0,12.2,24.8,...,4.2,5.0,8.7,1.3,0.3,3.0,3.2,33.9,125.0,117.0
5,2023-24,27.0,NYK,NBA,PG,56.0,56.0,2019.0,13.6,28.5,...,4.4,5.2,9.3,1.3,0.3,3.5,2.8,38.4,123.0,117.0


Season
Age
Tm
Lg
Pos
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
Season
Age
Tm
Lg
Pos
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
Season
Age
Tm
Lg
Pos
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
Unnamed: 29
ORtg
DRtg
Season
Age
Tm
Lg
Pos
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PTS
Unnamed: 29
ORtg
DRtg
('Unnamed: 0_level_0', 'Season')
('Unnamed: 1_level_0', 'Age')
('Unnamed: 2_level_0', 'Tm')
('Unnamed: 3_level_0', 'Lg')
('Unnamed: 4_level_0', 'Pos')
('Unnamed: 5_level_0', 'G')
('Unnamed: 6_level_0', 'MP')
('Unnamed: 7_level_0', 'Unnamed: 7_level_1')
('Shooting %', 'FG')
('Shooting %', '2P')
('Shooting %', '3P')
('Shooting %', 'eFG')
('Shooting %', 'FT')
('Shooting %', 'TS')
('Shooting %', 'FTr')
('Shooting %', '3PAr')
('Unnamed: 16_level_0', 'Unnamed: 16_level_1')
('League Shooting %', 'FG')
('League Shooting %', '2

KeyError: 'Season'

In [ ]:
## Play around with table categorization
time.sleep(5)
tableList = pd.read_html(str(brunson_soup_3.select("table")))
print([x.columns for x in tableList])

In [7]:
def scrapePlayerData(firstName, lastName):
    # Players with similar names have uncommon url structures
    curNumber = 1
    
    brunson_url = 'https://www.basketball-reference.com/players/'+lastName[0].lower()+'/'+lastName[:5].lower()+firstName[:2].lower()+'0'+str(curNumber)+'.html'
    
    # Use requests to retrieve data from a given URL
    brunson_response = requests.get(brunson_url)

    # Parse the whole HTML page using BeautifulSoup
    brunson_soup = BeautifulSoup(brunson_response.text, 'html.parser')

    # Title of the parsed page
    # brunson_soup.title.string

    # Find instance of paragraph markers for height and weight scraping
    strongMarkersBrunson = brunson_soup.find_all('strong')

    # unidecode removes accents from letters (helpful for international players)
    while unidecode(strongMarkersBrunson[0].string.lower()) != unidecode(firstName.lower() + " " + lastName.lower()):
        brunson_url = 'https://www.basketball-reference.com/players/'+lastName[0].lower()+'/'+lastName[:5].lower()+firstName[:2].lower()+'0'+str(curNumber)+'.html'
    
        # Use requests to retrieve data from a given URL
        brunson_response = requests.get(brunson_url)
        
        # Basketball reference has maximum of 20 requests per minute, so this will ensure we stay below that limit
        time.sleep(5)

        # Parse the whole HTML page using BeautifulSoup
        brunson_soup = BeautifulSoup(brunson_response.text, 'html.parser')

        # Title of the parsed page
        brunson_soup.title.string

        # Find instance of paragraph markers for height and weight scraping
        strongMarkersBrunson = brunson_soup.find_all('strong')
        
        
        curNumber+=1
        
        if curNumber > 9:
            return {"success":False, "Height":float("NaN"), "Weight":float("NaN"), 'shooting_df':float("NaN"), 'advanced_stat_df':float("NaN"),'per_100poss_df':float("NaN")}
            break


    # Basketball reference has maximum of 20 requests per minute, so this will ensure we stay below that limit
    time.sleep(3)



    # Create a dictionary to cache all of the player statistics
    brunsonInfoDict = {}

    print(strongMarkersBrunson[0].string.lower()+" found successfully!")


    # Find instance of paragraph markers for height and weight scraping
    paraMarkersBrunson = brunson_soup.find_all('p')

    # Basketball reference has maximum of 20 requests per minute, so this will ensure we stay below that limit
    time.sleep(3)

    for sectionNum in range(len(paraMarkersBrunson)):

        # Find Height data in cm's (standard for all player pages)
        if paraMarkersBrunson[sectionNum].string != None and "tall" in paraMarkersBrunson[sectionNum].string:
            # All centimeter height values for NBA players are in triple digits so the format is constant
            print("Height: ", paraMarkersBrunson[sectionNum].string[-13:-9].strip())
            brunsonInfoDict['Height'] = int(paraMarkersBrunson[sectionNum].string[-13:-9].strip())
            
        
        # Find Weight data in lbs (standard for all player pages)
        if paraMarkersBrunson[sectionNum].string != None and "weigh" in paraMarkersBrunson[sectionNum].string:
            # All kg weight values for NBA players are in double or triple digits
            weight = paraMarkersBrunson[sectionNum].string[paraMarkersBrunson[sectionNum].string.index('(')+1:paraMarkersBrunson[sectionNum].string.index(')')-3].strip()
            if weight[0] == '(':
                weight = weight[1:]
            brunsonInfoDict['Weight'] = int(weight.strip())
            
            # Passes 3 digit and 2 digit weight case in kg (Walker Kessler and Jalen Brunson)
            print("Weight: ", weight)
            
    print(brunsonInfoDict)   # Height and Weight Successfully Scraped

    # Basketball reference has maximum of 20 requests per minute, so this will ensure we stay below that limit
    time.sleep(3)

    ### PER 100 POSSESSIONS DATA
    # Attempt to extract data from the tables (Per 100 possesions table first)
    brunson_soup_2 = BeautifulSoup(requests.get(brunson_url).content, "lxml")
    brunson_soup_3 = BeautifulSoup("\n".join(brunson_soup_2.find_all(text=Comment)), "lxml")
    brunson_per_poss_df = pd.read_html(str(brunson_soup_3.select_one("table#per_poss")))[0]
    brunson_per_poss_df = brunson_per_poss_df.loc[:list(brunson_per_poss_df['Season']).index('Career')-1].dropna(axis=1, how='all')

    display(brunson_per_poss_df) # Prints the full dataframe correctly

    brunsonInfoDict['per_100poss_df'] = brunson_per_poss_df


    # Basketball reference has maximum of 20 requests per minute, so this will ensure we stay below that limit
    time.sleep(3)

    ### SHOOTING DATA
    # Attempt to extract data from the tables (Shooting table next)
    brunson_shooting_df = pd.read_html(str(brunson_soup_3.select_one("table#shooting")))[0]
    brunson_shooting_df.columns = [x[0].strip() + " ("+x[1].strip()+")" if "named" not in x[0].strip() else x[1].strip() for x in brunson_shooting_df.columns]
    brunson_shooting_df = brunson_shooting_df.loc[:list(brunson_shooting_df['Season']).index('Career')-1].dropna(axis=1, how='all')

    display(brunson_shooting_df) # Prints the full dataframe correctly
        
    brunsonInfoDict['shooting_df'] = brunson_shooting_df
    
    brunsonInfoDict['success'] = True
    
    print("Successfully Completed "+firstName+" "+lastName)
    
    return brunsonInfoDict
        
                

In [8]:
# Use this as player model because he has played the most years in the NBA, so we can fill with 0's
carterDict = scrapePlayerData('Vince','Carter')

vince carter found successfully!
Height:  198
Weight:  99
{'Height': 198, 'Weight': 99}


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,1998-99,22.0,TOR,NBA,SF,50.0,49.0,1760.0,10.6,23.5,...,5.8,8.7,4.6,1.7,2.4,3.4,4.3,28.1,107.0,104.0
1,1999-00,23.0,TOR,NBA,SF,82.0,82.0,3126.0,13.1,28.2,...,5.4,7.9,5.3,1.8,1.5,3.0,4.4,35.0,112.0,105.0
2,2000-01,24.0,TOR,NBA,SF,75.0,75.0,2979.0,13.5,29.3,...,4.3,7.4,5.2,2.0,1.5,3.0,3.6,36.7,114.0,104.0
3,2001-02,25.0,TOR,NBA,SG,60.0,60.0,2385.0,12.7,29.6,...,4.0,7.1,5.4,2.1,1.0,3.5,4.3,33.6,107.0,105.0
4,2002-03,26.0,TOR,NBA,SG,43.0,42.0,1471.0,12.9,27.6,...,4.7,6.8,5.2,1.7,1.5,2.7,4.4,32.1,109.0,108.0
5,2003-04,27.0,TOR,NBA,SG,73.0,73.0,2785.0,12.1,28.9,...,5.0,6.9,6.9,1.7,1.3,4.4,4.2,32.7,99.0,102.0
6,2004-05,28.0,TOT,NBA,"SF,SG",77.0,76.0,2828.0,13.1,29.1,...,5.6,7.6,6.2,2.1,0.9,3.2,4.6,35.6,109.0,104.0
7,2004-05,28.0,TOR,NBA,SG,20.0,20.0,608.0,10.6,25.7,...,3.2,5.6,5.2,2.1,1.3,1.9,4.8,27.0,102.0,109.0
8,2004-05,28.0,NJN,NBA,SF,57.0,56.0,2220.0,13.9,30.1,...,6.2,8.1,6.5,2.0,0.8,3.5,4.5,38.1,110.0,103.0
9,2005-06,29.0,NJN,NBA,SG,79.0,79.0,2906.0,12.0,27.9,...,6.0,8.5,6.2,1.7,1.0,3.9,4.3,35.1,107.0,102.0


,Season,Age,Tm,Lg,Pos,G,MP,FG%,Dist.,% of FGA by Distance (2P),...,FG% by Distance (16-3P),FG% by Distance (3P),% of FG Ast'd (2P),% of FG Ast'd (3P),Dunks (%FGA),Dunks (#),Corner 3s (%3PA),Corner 3s (3P%),Heaves (Att.),Heaves (#)
0,1998-99,22.0,TOR,NBA,SF,50.0,1760.0,0.450,11.7,0.914,...,0.419,0.288,0.564,0.842,0.097,69.0,0.121,0.375,2.0,0.0
1,1999-00,23.0,TOR,NBA,SF,82.0,3126.0,0.465,13.7,0.861,...,0.433,0.403,0.541,0.821,0.082,131.0,0.131,0.516,5.0,0.0
2,2000-01,24.0,TOR,NBA,SF,75.0,2979.0,0.460,14.1,0.760,...,0.428,0.408,0.558,0.772,0.028,44.0,0.169,0.403,1.0,0.0
3,2001-02,25.0,TOR,NBA,SG,60.0,2385.0,0.428,14.1,0.761,...,0.376,0.387,0.425,0.851,0.051,59.0,0.176,0.418,1.0,0.0
4,2002-03,26.0,TOR,NBA,SG,43.0,1471.0,0.467,14.3,0.828,...,0.395,0.344,0.506,0.822,0.058,43.0,0.183,0.333,0.0,0.0
5,2003-04,27.0,TOR,NBA,SG,73.0,2785.0,0.417,13.7,0.833,...,0.380,0.383,0.384,0.860,0.039,52.0,0.243,0.458,3.0,0.0
6,2004-05,28.0,TOT,NBA,"SF,SG",77.0,2828.0,0.452,12.8,0.797,...,0.430,0.406,0.508,0.835,0.051,70.0,0.115,0.361,2.0,0.0
7,2004-05,28.0,TOR,NBA,SG,20.0,608.0,0.411,13.6,0.805,...,0.440,0.322,0.495,0.842,0.036,11.0,0.203,0.250,0.0,0.0
8,2004-05,28.0,NJN,NBA,SF,57.0,2220.0,0.462,12.7,0.795,...,0.428,0.425,0.511,0.833,0.055,59.0,0.094,0.417,2.0,0.0
9,2005-06,29.0,NJN,NBA,SG,79.0,2906.0,0.430,12.6,0.758,...,0.414,0.341,0.441,0.744,0.062,87.0,0.169,0.371,1.0,0.0


Successfully Completed Vince Carter


In [4]:
# Import all drafted players since 1989
draftedPlayersDF = pd.read_csv("nbaplayersdraft.csv")

# Filter out players without 3 years of experience past 2004 (non contract extension eligible)
draftedPlayersDF = draftedPlayersDF[draftedPlayersDF['years_active']!=float("NaN")][draftedPlayersDF['years_active']>=3][draftedPlayersDF['year']>=1996][draftedPlayersDF['year']<=2020].reset_index()

draftedPlayersDF = draftedPlayersDF[['year', 'rank', 'overall_pick', 'team', 'player','years_active']].astype('object')

draftedPlayersDF['height'] = -1
draftedPlayersDF['weight'] = -1

for col in carterDict['shooting_df'].columns:
    for yearVal in range(len(carterDict['shooting_df'])):
            draftedPlayersDF[col + " year "+ str(yearVal)] = -1
    
for col in carterDict['per_100poss_df'].columns:
    for yearVal in range(len(carterDict['per_100poss_df'])):
            draftedPlayersDF[col + " year "+ str(yearVal)] = -1
    

print(draftedPlayersDF.columns)


display(draftedPlayersDF)

    

<ipython-input-4-b71a9d58a584>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  draftedPlayersDF = draftedPlayersDF[draftedPlayersDF['years_active']!=float("NaN")][draftedPlayersDF['years_active']>=3][draftedPlayersDF['year']>=1996][draftedPlayersDF['year']<=2020].reset_index()
<ipython-input-4-b71a9d58a584>:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  draftedPlayersDF[col + " year "+ str(yearVal)] = -1
<ipython-input-4-b71a9d58a584>:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  draftedPlayersDF[col + " year "+ str(yearVal)] = -1


Index(['year', 'rank', 'overall_pick', 'team', 'player', 'years_active',
       'height', 'weight', 'Season year 0', 'Season year 1',
       ...
       'DRtg year 16', 'DRtg year 17', 'DRtg year 18', 'DRtg year 19',
       'DRtg year 20', 'DRtg year 21', 'DRtg year 22', 'DRtg year 23',
       'DRtg year 24', 'DRtg year 25'],
      dtype='object', length=1360)


,year,rank,overall_pick,team,player,years_active,height,weight,Season year 0,Season year 1,...,DRtg year 16,DRtg year 17,DRtg year 18,DRtg year 19,DRtg year 20,DRtg year 21,DRtg year 22,DRtg year 23,DRtg year 24,DRtg year 25
0,1996,1,1,PHI,Allen Iverson,14.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1996,2,2,TOR,Marcus Camby,17.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1996,3,3,VAN,Shareef Abdur-Rahim,12.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1996,4,4,MIL,Stephon Marbury,13.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,1996,5,5,MIN,Ray Allen,18.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,2019,49,49,SAS,Quinndary Weatherspoon,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
952,2019,51,51,BOS,Tremont Waters,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
953,2019,52,52,CHO,Jalen McDaniels,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
954,2019,55,55,NYK,Kyle Guy,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
indexTemp = 2

draftedPlayersDF = draftedPlayersDF.copy()

for name in draftedPlayersDF['player'][3:7]:
    firstName,lastName = name.strip().split()[:2]
    print(firstName, lastName)
    
    # Remove periods for initials and the like within names
    firstName = unidecode(''.join([x for x in firstName if x.isalpha()]).strip())
    lastName = unidecode(''.join([x for x in lastName if x.isalpha()]).strip())
    

        
    curPlayerDict = scrapePlayerData(firstName,lastName)

    if curPlayerDict['success'] == False:
        draftedPlayersDF.loc[indexTemp,'height'] = float('NaN')
        draftedPlayersDF.loc[indexTemp,'weight'] = float('NaN')
        print(firstName+' '+lastName+' failed scraping...')
        continue


    draftedPlayersDF.loc[indexTemp,'height'] = curPlayerDict['Height']
    draftedPlayersDF.loc[indexTemp,'weight'] = curPlayerDict['Weight']

    for col in curPlayerDict['shooting_df'].columns:
        for yearVal in range(len(curPlayerDict['shooting_df'])):
            draftedPlayersDF.loc[indexTemp,col + " year "+ str(yearVal)] = curPlayerDict['shooting_df'].loc[yearVal,col]

    for col in curPlayerDict['per_100poss_df'].columns:
        for yearVal in range(len(curPlayerDict['per_100poss_df'])):
            draftedPlayersDF.loc[indexTemp, col + " year "+ str(yearVal)] = curPlayerDict['per_100poss_df'].loc[yearVal,col]
   
    
    indexTemp+=1
    
display(draftedPlayersDF)

Stephon Marbury
stephon marbury found successfully!
Height:  188
Weight:  81
{'Height': 188, 'Weight': 81}


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,1996-97,19.0,MIN,NBA,PG,67.0,64.0,2324.0,8.1,19.8,...,3.0,4.2,11.9,1.5,0.4,4.8,3.6,24.1,105.0,111.0
1,1997-98,20.0,MIN,NBA,PG,82.0,81.0,3112.0,8.5,20.6,...,2.9,3.8,11.7,1.7,0.1,4.3,3.7,24.2,105.0,111.0
2,1998-99,21.0,TOT,NBA,PG,49.0,49.0,1895.0,10.5,24.5,...,2.9,3.9,12.1,1.6,0.2,4.5,3.5,29.0,108.0,107.0
3,1998-99,21.0,MIN,NBA,PG,18.0,18.0,661.0,9.9,24.3,...,3.6,4.9,13.3,2.3,0.4,4.4,3.5,25.4,101.0,103.0
4,1998-99,21.0,NJN,NBA,PG,31.0,31.0,1234.0,10.8,24.6,...,2.6,3.4,11.5,1.3,0.1,4.6,3.4,30.8,111.0,109.0
5,1999-00,22.0,NJN,NBA,PG,74.0,74.0,2881.0,10.2,23.7,...,3.2,4.3,11.2,2.0,0.3,4.9,3.5,29.5,107.0,109.0
6,2000-01,23.0,NJN,NBA,PG,67.0,67.0,2557.0,11.5,26.1,...,3.1,4.2,10.3,1.6,0.1,4.0,3.1,32.7,110.0,109.0
7,2001-02,24.0,PHO,NBA,PG,82.0,80.0,3187.0,10.3,23.3,...,3.1,4.4,11.0,1.3,0.2,4.7,3.1,27.6,106.0,109.0
8,2002-03,25.0,PHO,NBA,PG,81.0,81.0,3240.0,10.9,24.8,...,3.4,4.3,10.6,1.7,0.3,4.3,3.2,29.2,108.0,106.0
9,2003-04,26.0,TOT,NBA,PG,81.0,81.0,3254.0,9.7,22.5,...,3.3,4.3,11.7,2.1,0.1,4.0,2.8,26.6,108.0,107.0


,Season,Age,Tm,Lg,Pos,G,MP,FG%,Dist.,% of FGA by Distance (2P),...,FG% by Distance (16-3P),FG% by Distance (3P),% of FG Ast'd (2P),% of FG Ast'd (3P),Dunks (%FGA),Dunks (#),Corner 3s (%3PA),Corner 3s (3P%),Heaves (Att.),Heaves (#)
0,1996-97,19.0,MIN,NBA,PG,67.0,2324.0,0.408,14.9,0.669,...,0.454,0.354,0.253,0.392,0.016,10.0,0.101,0.483,6.0,0.0
1,1997-98,20.0,MIN,NBA,PG,82.0,3112.0,0.415,13.8,0.754,...,0.394,0.313,0.292,0.495,0.007,7.0,0.115,0.371,7.0,0.0
2,1998-99,21.0,TOT,NBA,PG,49.0,1895.0,0.428,14.4,0.777,...,0.434,0.335,0.288,0.439,0.011,9.0,0.076,0.400,1.0,0.0
3,1998-99,21.0,MIN,NBA,PG,18.0,661.0,0.408,13.6,0.872,...,0.433,0.205,0.397,0.375,0.007,2.0,0.128,0.400,1.0,0.0
4,1998-99,21.0,NJN,NBA,PG,31.0,1234.0,0.439,14.8,0.727,...,0.435,0.367,0.224,0.448,0.014,7.0,0.063,0.400,0.0,0.0
5,1999-00,22.0,NJN,NBA,PG,74.0,2881.0,0.432,14.0,0.823,...,0.428,0.283,0.260,0.470,0.010,10.0,0.064,0.200,5.0,0.0
6,2000-01,23.0,NJN,NBA,PG,67.0,2557.0,0.441,12.6,0.738,...,0.412,0.328,0.285,0.527,0.002,2.0,0.072,0.167,4.0,1.0
7,2001-02,24.0,PHO,NBA,PG,82.0,3187.0,0.442,11.0,0.824,...,0.397,0.286,0.292,0.563,0.004,4.0,0.052,0.308,3.0,0.0
8,2002-03,25.0,PHO,NBA,PG,81.0,3240.0,0.439,11.6,0.807,...,0.421,0.301,0.294,0.404,0.002,3.0,0.078,0.217,4.0,0.0
9,2003-04,26.0,TOT,NBA,PG,81.0,3254.0,0.431,13.3,0.802,...,0.382,0.318,0.250,0.356,0.003,4.0,0.040,0.273,5.0,0.0


Successfully Completed Stephon Marbury
Ray Allen
ray allen found successfully!
Height:  196
Weight:  92
{'Height': 196, 'Weight': 92}


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,1996-97,21.0,MIL,NBA,SG,82.0,81.0,2532.0,8.3,19.2,...,4.8,6.9,4.4,1.6,0.2,3.2,4.6,23.3,109.0,110.0
1,1997-98,22.0,MIL,NBA,SG,82.0,82.0,3287.0,9.1,21.3,...,4.5,6.6,5.8,1.8,0.2,4.3,4.0,25.9,106.0,108.0
2,1998-99,23.0,MIL,NBA,SG,50.0,50.0,1719.0,9.7,21.6,...,5.0,6.8,5.7,1.7,0.2,3.9,3.8,27.5,110.0,106.0
3,1999-00,24.0,MIL,NBA,SG,82.0,82.0,3070.0,10.8,23.8,...,4.7,6.1,5.2,1.9,0.3,3.1,3.2,30.5,115.0,110.0
4,2000-01,25.0,MIL,NBA,SG,82.0,82.0,3129.0,10.5,21.8,...,5.4,7.1,6.2,2.1,0.3,3.4,3.2,30.1,119.0,106.0
5,2001-02,26.0,MIL,NBA,SG,69.0,67.0,2525.0,11.2,24.2,...,4.9,6.6,5.7,1.9,0.4,3.4,3.3,31.7,116.0,109.0
6,2002-03,27.0,TOT,NBA,SG,76.0,75.0,2880.0,11.1,25.4,...,5.3,7.1,6.2,1.9,0.3,3.7,4.1,31.9,111.0,108.0
7,2002-03,27.0,MIL,NBA,SG,47.0,46.0,1683.0,11.1,25.3,...,5.5,6.9,5.2,1.8,0.3,3.7,4.7,31.6,109.0,110.0
8,2002-03,27.0,SEA,NBA,SG,29.0,29.0,1197.0,11.2,25.5,...,5.2,7.4,7.7,2.1,0.1,3.7,3.2,32.3,114.0,105.0
9,2003-04,28.0,SEA,NBA,SG,56.0,56.0,2152.0,11.1,25.2,...,5.4,7.1,6.6,1.8,0.3,3.9,3.3,31.9,112.0,110.0


...

Successfully Completed Antoine Walker
Lorenzen Wright
lorenzen wright found successfully!
Height:  211
Weight:  102
{'Height': 211, 'Weight': 102}


,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,ORtg,DRtg
0,1996-97,21.0,LAC,NBA,C,77.0,51.0,1936.0,6.4,13.3,...,7.2,12.8,1.3,1.3,1.6,2.1,5.7,15.2,106.0,108.0
1,1997-98,22.0,LAC,NBA,C,69.0,38.0,2067.0,6.1,13.6,...,10.7,15.2,1.4,1.4,2.2,2.0,6.0,15.7,104.0,107.0
2,1998-99,23.0,LAC,NBA,C,48.0,15.0,1135.0,5.6,12.3,...,10.4,17.1,1.6,1.2,1.7,2.3,7.7,15.1,110.0,107.0
3,1999-00,24.0,ATL,NBA,C,75.0,0.0,1205.0,7.8,15.7,...,8.2,13.2,0.9,1.3,1.7,2.9,8.8,19.5,106.0,106.0
4,2000-01,25.0,ATL,NBA,C,71.0,46.0,1988.0,9.5,21.3,...,9.3,14.0,2.3,1.1,1.7,3.3,6.1,23.1,99.0,103.0
5,2001-02,26.0,MEM,NBA,C,43.0,33.0,1251.0,9.4,20.5,...,11.6,17.1,1.9,1.3,1.0,3.0,5.3,21.7,96.0,104.0
6,2002-03,27.0,MEM,NBA,C,70.0,49.0,1982.0,8.5,18.7,...,9.4,13.8,2.1,1.3,1.4,2.9,5.5,20.8,99.0,106.0
7,2003-04,28.0,MEM,NBA,C,65.0,46.0,1674.0,8.1,18.4,...,9.4,14.0,2.2,1.4,1.8,2.4,6.0,19.1,100.0,101.0
8,2004-05,29.0,MEM,NBA,C,80.0,77.0,2287.0,7.6,16.2,...,10.3,14.5,2.1,1.4,1.6,2.4,6.0,18.3,103.0,101.0
9,2005-06,30.0,MEM,NBA,C,78.0,58.0,1689.0,6.4,13.4,...,8.3,13.0,1.6,1.7,1.5,2.2,6.6,15.0,103.0,100.0


,Season,Age,Tm,Lg,Pos,G,MP,FG%,Dist.,% of FGA by Distance (2P),...,FG% by Distance (16-3P),FG% by Distance (3P),% of FG Ast'd (2P),% of FG Ast'd (3P),Dunks (%FGA),Dunks (#),Corner 3s (%3PA),Corner 3s (3P%),Heaves (Att.),Heaves (#)
0,1996-97,21.0,LAC,NBA,C,77.0,1936.0,0.481,8.2,0.992,...,0.375,0.250,0.515,1.0,0.081,39.0,0.250,0.0,0.0,0.0
1,1997-98,22.0,LAC,NBA,C,69.0,2067.0,0.445,8.5,0.996,...,0.404,0.000,0.676,NaN,0.092,46.0,0.000,NaN,0.0,0.0
2,1998-99,23.0,LAC,NBA,C,48.0,1135.0,0.458,7.9,0.996,...,0.417,0.000,0.538,NaN,0.077,19.0,0.000,NaN,0.0,0.0
3,1999-00,24.0,ATL,NBA,C,75.0,1205.0,0.499,8.0,0.992,...,0.353,0.333,0.536,1.0,0.089,32.0,0.333,1.0,1.0,0.0
4,2000-01,25.0,ATL,NBA,C,71.0,1988.0,0.448,7.8,0.998,...,0.341,0.000,0.617,NaN,0.058,43.0,0.500,0.0,0.0,0.0
5,2001-02,26.0,MEM,NBA,C,43.0,1251.0,0.459,6.4,0.996,...,0.417,0.000,0.682,NaN,0.082,36.0,0.000,NaN,0.0,0.0
6,2002-03,27.0,MEM,NBA,C,70.0,1982.0,0.454,7.9,0.993,...,0.405,0.000,0.665,NaN,0.068,44.0,0.000,NaN,0.0,0.0
7,2003-04,28.0,MEM,NBA,C,65.0,1674.0,0.439,8.4,0.991,...,0.343,0.000,0.747,NaN,0.072,38.0,0.000,NaN,1.0,0.0
8,2004-05,29.0,MEM,NBA,C,80.0,2287.0,0.469,7.1,0.996,...,0.387,0.000,0.734,NaN,0.097,61.0,0.000,NaN,1.0,0.0
9,2005-06,30.0,MEM,NBA,C,78.0,1689.0,0.478,6.2,0.995,...,0.306,0.000,0.696,NaN,0.113,44.0,0.000,NaN,0.0,0.0


Successfully Completed Lorenzen Wright


,year,rank,overall_pick,team,player,years_active,height,weight,Season year 0,Season year 1,...,DRtg year 16,DRtg year 17,DRtg year 18,DRtg year 19,DRtg year 20,DRtg year 21,DRtg year 22,DRtg year 23,DRtg year 24,DRtg year 25
0,1996,1,1,PHI,Allen Iverson,14.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1996,2,2,TOR,Marcus Camby,17.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1996,3,3,VAN,Shareef Abdur-Rahim,12.0,188,81,1996-97,1997-98,...,106,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1996,4,4,MIL,Stephon Marbury,13.0,196,92,1996-97,1997-98,...,104,101,107,109,-1,-1,-1,-1,-1,-1
4,1996,5,5,MIN,Ray Allen,18.0,206,101,1996-97,1997-98,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,2019,49,49,SAS,Quinndary Weatherspoon,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
952,2019,51,51,BOS,Tremont Waters,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
953,2019,52,52,CHO,Jalen McDaniels,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
954,2019,55,55,NYK,Kyle Guy,3.0,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [ ]:
draftedPlayersDF[draftedPlayersDF['player'] == "Jordan Poole"]['height']